In [17]:
import os
import pandas as pd
import pickle
from ast import literal_eval
import logging
import argparse
from simpletransformers.ner import (
    NERArgs,
    NERModel,
)

In [2]:
BIO_type = 1

labelset = ["B-p", "I-p", "B-n", "I-n", "O"]
type_ls = ['-p', '-n']

In [3]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

pos = pd.read_csv('pos_sent.csv')
print(len(pos))
col_name = pos.columns[6+BIO_type]
pos = pos.dropna(axis=0, subset=[col_name])
print(len(pos))

3149
3149


In [4]:
data = []
for i in range(len(pos)):
    words = pos.iloc[i, 1].split(' ')
    tags = literal_eval(pos.iloc[i, 6+BIO_type])
    for j in range(len(words)):
        data.append([i, words[j], tags[j]])
df = pd.DataFrame(data, columns=['sentence_id', 'words', 'labels'])

In [5]:
model_args = NERArgs()
model_args.reprocess_input_data = True
model_args.save_model_every_epoch = False
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.manual_seed = 1
model_args.use_multiprocessing = True
model_args.do_lower_case = False

In [6]:
def phrase_F1(ref, pred):
    return 0

# Create a TransformerModel
model = NERModel(
    "xlmroberta",
    "../ner/Noutputs/best_model",
    labels=labelset,
    args=model_args,
)

In [7]:
result, model_outputs, pred_label = model.eval_model(df, F1_score=phrase_F1)
# 'pred_label' is a list, where each element is the list of predicted labels for one sentence

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/3149 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/394 [00:00<?, ?it/s]

/opt/conda/envs/hl/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
INFO:simpletransformers.ner.ner_model:{'eval_loss': 2.7530998868990673, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'F1_score': 0}


In [20]:
p=[str(l) for l in pred_label]

In [21]:
pos = pd.read_csv('pos_sent.csv')
pos['BIO_1']=p

In [ ]:
type_ls = ['-p', '-n']
def get_entity_spans(idx, ls):
    # given a sequence of BIO taggs, get the list of tuples representing spans of entities
    spans = [[],[]]
    for t in range(2):
        st, ed = 0, 0
        for i in range(len(ls)):
            #st, ed = 0, 0
            if ls[i] == 'B' + type_ls[t]:
                st, ed = i, i + 1
                for j in range(i+1, len(ls)):
                    if ls[j] == 'I' + type_ls[t]:
                        ed += 1
                    else:
                        break
                phrase=' '.join(pos.loc[idx,'text'].split(' ')[st:ed])
                tup=(phrase, (st, ed))
                spans[t].append(tup)
    spans[0] = sorted(spans[0], key=lambda x: x[0])
    spans[0] = sorted(spans[1], key=lambda x: x[0])
    return spans

In [ ]:
def get_entity_spans(k, ls):
    spans = [[],[]]
    for i in range(len(ls)):
        if ls[i]=='B-p':
            for j in range(i+1,len(ls)):
                if ls[j]!='I-p':
                    phrase=' '.join(pos.loc[k,'text'].split(' ')[i:j])
                    tup=(phrase, (i, j))
                    spans[0].append(tup)
                    break
        elif ls[i]=='B-n':
            for j in range(i+1,len(ls)):
                if ls[j]!='I-n':
                    phrase=' '.join(pos.loc[k,'text'].split(' ')[i:j])
                    tup=(phrase, (i, j))
                    spans[1].append(tup)
                    break
    return spans
                    

In [ ]:
all=[]
for k in range(len(pred_label)):
    all.append(get_entity_spans(k, pred_label[k]))
all


In [ ]:
dt=pd.read_csv('pos_sent.csv')
#dt=dt['labels','text']
a=pd.DataFrame(all)
a.columns=['predicates', 'subj/obj']
a=pd.concat([dt[['labels','text']],a],axis=1)
a['triple_A']='[]'
a['triple_B']='[]'
a['triple_C']='[]'
a['triple_D']='[]'
a=a.reset_index(drop=True)

In [ ]:
a.to_csv('triples.csv',index=False)